In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
import numpy as np
import sys

In [3]:
# Función para cargar y procesar una hoja
def cargar_hoja(archivo, hoja, skiprows, moneda):
    df = pd.read_excel(archivo, sheet_name=hoja, skiprows=skiprows, usecols=lambda x: 'Unnamed' not in x)
    df["Moneda"] = moneda
    return df.fillna(method='ffill')

# Cargar todas las hojas
df_eeff_mn = cargar_hoja("Boletín Bancos MN.xlsm", "1. EEFF", 23, "MN")
df_eeff_me = cargar_hoja("Boletín Bancos ME.xlsm", "1. EEFF", 23, "ME")

df_cartera_mn = cargar_hoja("Boletín Bancos MN.xlsm", "3. Cartera", 23, "MN")
df_cartera_me = cargar_hoja("Boletín Bancos ME.xlsm", "3. Cartera", 23, "ME")

df_sector_mn = cargar_hoja("Boletín Bancos MN.xlsm", "5. Cred. por sector", 21, "MN")
df_sector_me = cargar_hoja("Boletín Bancos ME.xlsm", "5. Cred. por sector", 21, "ME")

df_actividad_mn = cargar_hoja("Boletín Bancos MN.xlsm", "6. Cred. por acti", 17, "MN")
df_actividad_me = cargar_hoja("Boletín Bancos ME.xlsm", "6. Cred. por acti", 17, "ME")

# Unir MN y ME por tipo
df_eeff = pd.concat([df_eeff_mn, df_eeff_me], ignore_index=True)
df_cartera = pd.concat([df_cartera_mn, df_cartera_me], ignore_index=True)
df_sector = pd.concat([df_sector_mn, df_sector_me], ignore_index=True)
df_actividad = pd.concat([df_actividad_mn, df_actividad_me], ignore_index=True)

# Guardar en un nuevo archivo Excel con varias hojas
with pd.ExcelWriter("Boletin_Procesado.xlsx", engine='openpyxl') as writer:
    df_eeff.to_excel(writer, sheet_name="EEFF", index=False)
    df_cartera.to_excel(writer, sheet_name="Cartera", index=False)
    df_sector.to_excel(writer, sheet_name="Cred_por_sector", index=False)
    df_actividad.to_excel(writer, sheet_name="Cred_por_actividad", index=False)

C:\Users\Mathias\AppData\Local\Temp\ipykernel_22672\1971185955.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return df.fillna(method='ffill')
C:\Users\Mathias\AppData\Local\Temp\ipykernel_22672\1971185955.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return df.fillna(method='ffill')
C:\Users\Mathias\AppData\Local\Temp\ipykernel_22672\1971185955.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return df.fillna(method='ffill')
C:\Users\Mathias\AppData\Local\Temp\ipykernel_22672\1971185955.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return df.fillna(method='ffill')
C:\Users\Mathias\AppData\Local\Temp\ipyk